---

* 출처: LangChain 공식 문서 또는 해당 교재명
* 원본 URL: https://smith.langchain.com/hub/teddynote/summary-stuff-documents

---

### **02. `RAGAS`를 활용한 `평가`**

#### **1) `RAGAS를 활용한 평가`**

* 참고: [RAGAS](https://cal.com/docs/developing/introduction)

  * **`API v1`**: [API v1](https://cal.com/docs/api-reference/v1/introduction)

  * **`API v2`**: [API v2](https://cal.com/docs/api-reference/v2/introduction)

* 사전에 `VS Code` 터미널에 설치할 것

```bash

        pip install -qU faiss-cpu ragas


```

##### **`➀ 기본설정`**

In [ ]:
import os
import logging
import warnings

warnings.filterwarnings("ignore")
os.environ["TOKENIZERS_PARALLELISM"] = "false"          # 토크나이저 병렬 처리 경고 억제
os.environ["TQDM_DISABLE"] = "1"                        # 진행바 비활성화
logging.disable(logging.WARNING)                        # 파이썬 로깅 레벨 조정 (WARNING 이하 출력 억제)

from dotenv import load_dotenv
load_dotenv()

print("✅ 환경 설정 완료")

<small>

* ✅ 환경 설정 완료

##### **`➁ LangSmith 설정`**

In [ ]:
from langsmith import Client
from langsmith import traceable

import os

# LangSmith 환경 변수 확인

print("\n--- LangSmith 환경 변수 확인 ---")
langchain_tracing_v2 = os.getenv('LANGCHAIN_TRACING_V2')
langchain_project = os.getenv('LANGCHAIN_PROJECT')
langchain_api_key_status = "설정됨" if os.getenv('LANGCHAIN_API_KEY') else "설정되지 않음" # API 키 값은 직접 출력하지 않음

if langchain_tracing_v2 == "true" and os.getenv('LANGCHAIN_API_KEY') and langchain_project:
    print(f"✅ LangSmith 추적 활성화됨 (LANGCHAIN_TRACING_V2='{langchain_tracing_v2}')")
    print(f"✅ LangSmith 프로젝트: '{langchain_project}'")
    print(f"✅ LangSmith API Key: {langchain_api_key_status}")
    print("  -> 이제 LangSmith 대시보드에서 이 프로젝트를 확인해 보세요.")
else:
    print("❌ LangSmith 추적이 완전히 활성화되지 않았습니다. 다음을 확인하세요:")
    if langchain_tracing_v2 != "true":
        print(f"  - LANGCHAIN_TRACING_V2가 'true'로 설정되어 있지 않습니다 (현재: '{langchain_tracing_v2}').")
    if not os.getenv('LANGCHAIN_API_KEY'):
        print("  - LANGCHAIN_API_KEY가 설정되어 있지 않습니다.")
    if not langchain_project:
        print("  - LANGCHAIN_PROJECT가 설정되어 있지 않습니다.")

<small>

* 셀 출력

    ```bash
    --- LangSmith 환경 변수 확인 ---
    ✅ LangSmith 추적 활성화됨 (LANGCHAIN_TRACING_V2='true')
    ✅ LangSmith 프로젝트: 'LangChain-prantice'
    ✅ LangSmith API Key: 설정됨
    -> 이제 LangSmith 대시보드에서 이 프로젝트를 확인해 보세요.
    ```

In [ ]:
# 필요한 패키지 임포트

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama
from langchain_huggingface import HuggingFaceEmbeddings

print("✅ 패키지 임포트 완료")

<small>

* ✅ 패키지 임포트 완료 - (`0.6s`)

---

##### **`➂ 교재 원본 파일로 로드해보기`**

* **`저장한 CSV 파일로부터 로드하기`**

  * `data/ragas_synthetic_dataset_original.csv` 파일 로드하기

In [ ]:
import pandas as pd

# 데이터 로드하기
df = pd.read_csv("../15_Evaluations/data/ragas_synthetic_dataset_original.csv")

# 일부 정보 읽어보기
df.head()

<small>

* **`df.head()`** - (`0.7s`)

  * ![df.head()](../15_Evaluations/assets/02_original_df_head.png)

In [ ]:
from datasets import Dataset

test_dataset = Dataset.from_pandas(df)

test_dataset

<small>

* **`test_dataset`** - (`0.4s`)

```python

        Dataset({
            features: ['question', 'contexts', 'ground_truth', 'evolution_type', 'metadata', 'episode_done'],
            num_rows: 10
            })

```

  * `test_dataset` 확인해보기

    * ![test_dataset](../15_Evaluations/assets/02_original_test_dataset.png)

In [ ]:
import ast                                  # 문자열을 파이썬 리터럴로 안전하게 평가하는 데 사용

# 주어진 예제 딕셔너리의 'contexts' 키에 저장된 문자열을 파이썬 리스트로 변환하는 함수
def convert_to_list(example):
    # 문자열 = 파이썬 리터럴로 변환함
    contexts = ast.literal_eval(example["contexts"])
    # 변환된 리스트 = 딕셔너리로 반환함
    return {"contexts": contexts}

# test_dataset의 각 요소에 convert_to_list 함수 적용하기
# map 함수 = 데이터셋의 각 행에 함수를 적용하는 데 사용
test_dataset = test_dataset.map(convert_to_list)

# 변환된 데이터셋을 출력하기
print(test_dataset)

<small>

* **`변환된 데이터셋`** - (`0.1s`)

```bash

        Map:   100%|████████████████████████████████████████| 10/10 [00:00<00:00, 362.11 examples/s]

```

```python

        Dataset({
            features: ['question', 'contexts', 'ground_truth', 'evolution_type', 'metadata', 'episode_done'],
            num_rows: 10
        })

```

In [ ]:
test_dataset[1]["contexts"]

<small>

```bash

['Ⅱ\n. 주요 행사 일정\n행사명 행사 주요 개요\n- 미국 소비자기술 협회(CTA)가 주관하는 세계 최대 가전·IT·소\n비재 전시회로 5G, AR&VR, 디지털헬스, 교통·모빌리티 등\n주요 카테고리 중심으로 기업들이 최신의 기술 제품군을 전시\n- CTA 사피로 회장은 가장 주목받는 섹터로 AI를 조명하였으며,\n모든 산업을 포괄한다는 의미에서 ‘올 온(All on)’을 주제로 한\nCES 2024\n이번 전시에는 500곳 이상의 한국기업 참가 예정\n기간 장소 홈페이지\n2024.1.9~12 미국, 라스베가스 https://www.ces.tech/\n- 머신러닝 및 응용에 관한 국제 컨퍼런스(AIMLA 2024)는\n인공지능 및 머신러닝의 이론, 방법론 및 실용적 접근에 관한\n지식과 최신 연구 결과 공유\n- 이론 및 실무 측면에서 인공지능, 기계학습의 주요 분야를\n논의하고, 학계, 산업계의 연구자와 실무자들에게 해당 분\nAIMLA 2024\n야의 최첨단 개발 소식 공유\n기간 장소 홈페이지\nhttps://ccnet2024.org/aimla\n2024.1.27~28 덴마크, 코펜하겐\n/index\n- AI 발전 협회 컨퍼런스(AAAI)는 AI 연구를 촉진하고, AI 분야\n연구원, 실무자, 과학자, 학생 및 공학자 간 교류의 기회 제공\n- 컨퍼런스에서 AI 관련 기술 발표, 특별 트랙, 초청 연사,\nAAAI Conference\n워크숍, 튜토리얼, 포스터 세션, 주제 발표, 대회, 전시 프\non Artificial\n로그램 등 진행\nIntelligence\n기간 장소 홈페이지\nhttps://aaai.org/aaai-confere\n2024.2.20~27 캐나다, 밴쿠버\nnce/']

```

In [ ]:
# 단계 1: 문서 로드(Load Documents)
loader = PyMuPDFLoader("../15_Evaluations/data/SPRI_AI_Brief_2023년12월호_F.pdf")
docs = loader.load()                                                    # 0.3s

In [ ]:
# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)

In [ ]:
# 단계 3: 임베딩(Embedding) 생성
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

print("✅ 임베딩 생성 완료")

<small>

* ✅ 임베딩 생성 완료 - (`8.8s`)

In [ ]:
# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어 생성하기

vectorstore = FAISS.from_documents(
    documents=split_documents,
    embedding=embeddings
    )

print("✅ vectorstore 생성 완료")

<small>

* ✅ vectorstore 생성 완료 - (`5.1s`)

In [ ]:
# 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보 → 검색 및 생성
retriever = vectorstore.as_retriever()

print("✅ retriever 생성 완료")

In [ ]:
# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트 생성하기

prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 

#Context: 
{context}

#Question:
{question}

#Answer:"""
)

print("✅ prompt 생성 완료")

<small>

* ✅ prompt 생성 완료

In [ ]:
# 단계 7: 언어모델(LLM) 생성
llm=ChatOllama(
    model="llama3.2:3b",
    temperature=0.1,
)

print("✅ Ollama LLM 초기화 완료 (모델: llama3.2:3b)")

<small>

* ✅ Ollama LLM 초기화 완료 (모델: llama3.2:3b)

In [ ]:
# 단계 8: 체인(Chain) 생성

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("✅ Chain 생성 완료")

<small>

* ✅ Chain 생성 완료

---

* **`배치 데이터셋` 생성하기**

  * **`다량의 질문을 한 번에 처리할 때 용이함`**

  * [**`배치`**](https://wikidocs.net/233345): (`https://wikidocs.net/233345`)

In [ ]:
# 배치 데이터셋 생성하기

batch_dataset = [question for question in test_dataset["question"]]

batch_dataset[:3]

<small>

* **`batch_dataset[:3]`**

```python

        ['What specific recent developments in generative AI have been highlighted by TechRepublic in their November 2023 articles?',
        'What are the dates and location for CES 2024?',
        "What are the key aspects of AI global cooperation highlighted in the G7's approach to ethical considerations and regulatory frameworks for advanced AI systems?"]

```

In [ ]:
print(type(batch_dataset))              # <class 'list'>

In [ ]:
print(batch_dataset)

<small>

* **`print(batch_dataset)`**

<small> <small>

```python

        ['What specific recent developments in generative AI have been highlighted by TechRepublic in their November 2023 articles?', 'What are the dates and location for CES 2024?', "What are the key aspects of AI global cooperation highlighted in the G7's approach to ethical considerations and regulatory frameworks for advanced AI systems?", 'What measures are suggested to enhance data privacy in the context of advanced AI systems?', 'What were the main takeaways from the AI Safety Summit on global AI testing?', 'What links data provenance, model efficiency, and NLP accuracy?', 'What’s the MMLU benchmark, and how does it evaluate AI models like Tongyi Qianwen 2.0 for generative AI?', 'How does the MMLU benchmark assess AI models like Tongyi Qianwen 2.0?', 'How do changes in AI eval criteria affect LLMs?', 'What factors matter for ethical AI innovation?']

```

---

<small>

* before

* `batch_dataset[:3]`

```python

    ['What specific recent developments in generative AI have been highlighted by TechRepublic in their November 2023 articles?',
    'What are the dates and location for CES 2024?',
    "What are the key aspects of AI global cooperation highlighted in the G7's approach to ethical considerations and regulatory frameworks for advanced AI systems?"]

```

* `batch_dataset`

<small> <small>

```python

    ['What specific recent developments in generative AI have been highlighted by TechRepublic in their November 2023 articles?', 'What are the dates and location for CES 2024?', "What are the key aspects of AI global cooperation highlighted in the G7's approach to ethical considerations and regulatory frameworks for advanced AI systems?", 'What measures are suggested to enhance data privacy in the context of advanced AI systems?', 'What were the main takeaways from the AI Safety Summit on global AI testing?', 'What links data provenance, model efficiency, and NLP accuracy?', 'What’s the MMLU benchmark, and how does it evaluate AI models like Tongyi Qianwen 2.0 for generative AI?', 'How does the MMLU benchmark assess AI models like Tongyi Qianwen 2.0?', 'How do changes in AI eval criteria affect LLMs?', 'What factors matter for ethical AI innovation?']

```

---

* **`batch()` 호출 → 배치 데이터셋에 대한 답변 얻기**

In [ ]:
answer = chain.batch(batch_dataset)

answer[:3]

<small>

* **`try_4`** - (`5m 49.3s`)

```python

        ['According to the provided context, TechRepublic has highlighted that Samsung has developed a generative AI called "Samsung Gauss" and has revealed its research on it. Additionally, they mentioned that Samsung\'s Llama2 is being used in Qualcomm devices and Google Assistant is being applied to Google Pixel phones, which are competing with each other in the market.',
        'According to the retrieved context, the dates and location for CES 2024 are:\n\n* Dates: 2024.1.9~12\n* Location: 미국 (United States), 라스베가스 (Las Vegas)',
        'The key aspects of AI global cooperation highlighted in the G7\'s approach to ethical considerations and regulatory frameworks for advanced AI systems include:\n\n1. International Code of Conduct for Advanced AI Systems: The G7 has agreed on an international code of conduct for advanced AI systems, which includes provisions for identifying and mitigating risks, ensuring transparency and accountability, and promoting responsible AI development and use.\n2. Voluntary Adoption: The G7 is encouraging companies to voluntarily adopt the international code of conduct, with a focus on developing and deploying safe and secure AI systems.\n3. Risk Evaluation and Mitigation: The G7 is emphasizing the need for risk evaluation and mitigation measures to be taken during the development and deployment of advanced AI systems.\n4. Transparency and Accountability: The G7 is promoting transparency and accountability in AI decision-making, including the publication of model performance data and the establishment of mechanisms for addressing bias and errors.\n5. International Cooperation: The G7 is encouraging international cooperation on AI-related issues, including the sharing of best practices and the development of common standards.\n6. Regulatory Frameworks: The G7 is exploring regulatory frameworks to ensure that advanced AI systems are developed and deployed in a responsible and safe manner.\n7. Model Card: Three countries (France, Germany, and Italy) have proposed the use of a "model card" to regulate foundation models, which would require companies to provide information about their models and ensure compliance with regulations.\n\nThese aspects highlight the G7\'s commitment to promoting responsible AI development and use, while also acknowledging the need for international cooperation and regulatory frameworks to ensure that advanced AI systems are developed and deployed in a safe and secure manner.']

```

---

<small>

* **`try_1`** - (`5m 56.1s`)

<small> <small>

```python

    ['According to the provided context, TechRepublic has highlighted that Samsung has developed a generative AI called "Samsung Gauss" and has revealed its research on it. Additionally, they mentioned that Meta\'s Llama 2 is being used in Qualcomm devices and Google Assistant is being applied to Google Pixel phones, which are competing with Samsung\'s new device.',
    'According to the retrieved context, the dates and location for CES 2024 are:\n\n* Dates: 2024.1.9~12\n* Location: 미국 (United States), 라스베가스 (Las Vegas)',
    "The key aspects of AI global cooperation highlighted in the G7's approach to ethical considerations and regulatory frameworks for advanced AI systems include:\n\n1. International Code of Conduct for Advanced AI Systems: The G7 has agreed on an international code of conduct for advanced AI systems, which includes provisions for identifying and mitigating risks, ensuring transparency and accountability, and promoting responsible AI development and use.\n2. Voluntary Adoption: The G7 encourages companies to adopt the international code of conduct voluntarily, with a focus on self-regulation and industry-led initiatives.\n3. Risk Evaluation and Mitigation: The code requires companies to evaluate and mitigate risks associated with advanced AI systems, including those related to bias, transparency, and accountability.\n4. Transparency and Accountability: The code emphasizes the importance of transparency and accountability in AI development and use, including the disclosure of model performance and limitations.\n5. Collaboration and Cooperation: The G7 encourages collaboration and cooperation among governments, industry, and civil society to address AI-related challenges and promote responsible AI development and use.\n6. Information Sharing: The code promotes information sharing among countries and stakeholders to facilitate the development of effective regulatory frameworks and ensure consistent application of standards.\n7. Standardization: The G7 aims to establish common standards for advanced AI systems, including those related to model performance, bias, and transparency.\n\nThese key aspects highlight the G7's commitment to promoting responsible AI development and use through international cooperation, voluntary adoption, and industry-led initiatives."]

```

<small>

* **`try_2`** - (`5m 36.0s`)

  * `request_timeout=None,`, `format="json"` 옵션 추가했을 때의 답변 

<small> <small>

```python

        ['{"name": "Samsung Gauss", "text": "TechRepublic reported that Samsung Research has revealed a generative AI called \'Samsung Gauss\'."}',
        '{\n"기간": "2024.1.9~12",\n"장소": "미국, 라스베가스",\n"홈페이지": "https://www.ces.tech/"\n}',
        '{\n"위험 평가 및 완화": "G7은 AI 수명주기 전반에 걸쳐 위험을 평가 및 완화하는 조치를 채택하고, 취약점과 오용 사고, 오용 유형을 파악해 완화합니다.",\n"투명성과 책임성의 보장": "G7은 첨단 AI 시스템의 성능과 한계를 공개하고 적절하거나 부적절한 사용영역을 알리는 방법으로 투명성을 보장하고 책임성을 강화합니다.",\n"정보공유와 이해관계자 간 협력": "G7은 관련된 경우 해당 국가와 결과를 공유하며, 적절한 시기에 공동 표준 개발을 위해 노력하기로 합의합니다.",\n"기반모델과 생성 AI를 포함한 첨단 AI 시스템의 위험 식별과 완화": "G7은 기반모델과 생성 AI를 포함한 첨단 AI 시스템의 위험 식별과 완화를 위한 조치를 포함하여 행동강령을 마련합니다."\n}']

```

<small>

* **`try_3`** - (`5m 40.2s`)

<small> <small>

```python

    ['According to the retrieved context, TechRepublic has highlighted that Samsung has developed a generative AI called "Samsung Gauss" which is expected to compete with Meta\'s Llama2 and Google\'s Assistant on recent smartphones.',
    'According to the retrieved context, the dates and location for CES 2024 are:\n\n* Dates: 2024.1.9~12\n* Location: 미국 (United States), 라스베가스 (Las Vegas)',
    'The key aspects of AI global cooperation highlighted in the G7\'s approach to ethical considerations and regulatory frameworks for advanced AI systems include:\n\n1. International Code of Conduct for Advanced AI Systems: The G7 has agreed on an international code of conduct for advanced AI systems, which includes provisions for identifying and mitigating risks, ensuring transparency and accountability, and promoting responsible AI development and use.\n2. Voluntary Adoption: The G7 is encouraging voluntary adoption of the international code of conduct by companies developing advanced AI systems.\n3. Risk Evaluation and Mitigation: The G7 is requiring a risk evaluation and mitigation process for advanced AI systems, including the identification of potential risks and the implementation of measures to mitigate them.\n4. Transparency and Accountability: The G7 is promoting transparency and accountability in AI development and use, including the disclosure of information about AI systems and their limitations.\n5. Cooperation with International Organizations: The G7 is encouraging cooperation with international organizations, such as the United Nations, to promote global coordination on AI issues.\n6. Development of a Global Framework: The G7 is working towards developing a global framework for regulating advanced AI systems, which would provide a common set of standards and guidelines for countries to follow.\n7. Collaboration with Other Countries: The G7 is engaging in collaboration with other countries, including through the "Hyorishima AI Process", to promote global cooperation on AI issues.\n\nThese aspects highlight the G7\'s commitment to promoting responsible AI development and use, while also recognizing the need for international cooperation and coordination to address the challenges posed by advanced AI systems.']

```

---

In [ ]:
print(type(answer))                     # <class 'list'>

In [ ]:
print(answer)

<small>

* **`print(answer)`**

<small> <small>

```python

        ['According to the provided context, TechRepublic has highlighted that Samsung has developed a generative AI called "Samsung Gauss" and has revealed its research on it. Additionally, they mentioned that Samsung\'s Llama2 is being used in Qualcomm devices and Google Assistant is being applied to Google Pixel phones, which are competing with each other in the market.', 'According to the retrieved context, the dates and location for CES 2024 are:\n\n* Dates: 2024.1.9~12\n* Location: 미국 (United States), 라스베가스 (Las Vegas)', 'The key aspects of AI global cooperation highlighted in the G7\'s approach to ethical considerations and regulatory frameworks for advanced AI systems include:\n\n1. International Code of Conduct for Advanced AI Systems: The G7 has agreed on an international code of conduct for advanced AI systems, which includes provisions for identifying and mitigating risks, ensuring transparency and accountability, and promoting responsible AI development and use.\n2. Voluntary Adoption: The G7 is encouraging companies to voluntarily adopt the international code of conduct, with a focus on developing and deploying safe and secure AI systems.\n3. Risk Evaluation and Mitigation: The G7 is emphasizing the need for risk evaluation and mitigation measures to be taken during the development and deployment of advanced AI systems.\n4. Transparency and Accountability: The G7 is promoting transparency and accountability in AI decision-making, including the publication of model performance data and the establishment of mechanisms for addressing bias and errors.\n5. International Cooperation: The G7 is encouraging international cooperation on AI-related issues, including the sharing of best practices and the development of common standards.\n6. Regulatory Frameworks: The G7 is exploring regulatory frameworks to ensure that advanced AI systems are developed and deployed in a responsible and safe manner.\n7. Model Card: Three countries (France, Germany, and Italy) have proposed the use of a "model card" to regulate foundation models, which would require companies to provide information about their models and ensure compliance with regulations.\n\nThese aspects highlight the G7\'s commitment to promoting responsible AI development and use, while also acknowledging the need for international cooperation and regulatory frameworks to ensure that advanced AI systems are developed and deployed in a safe and secure manner.', 'I don\'t know the specific answer to your question based on the provided context. However, I can suggest that you may want to look into the documents related to "보건복지 부문" (Health and Welfare Department) or "혁신과 경쟁 촉진" (Innovation and Competition Promotion) sections in the PDF file, as they seem to be related to AI development and deployment strategies.', 'The main takeaways from the AI Safety Summit on global AI testing include:\n\n1. The importance of cooperation among nations, international organizations, companies, civil society, and academia in ensuring AI safety.\n2. The need for transparency and accountability in AI development, particularly from advanced AI systems developers.\n3. The establishment of a framework for evaluating and testing AI systems to ensure their safety and security.\n4. The creation of a global network of research institutions, governments, and industries to share knowledge and best practices on AI safety.\n5. The commitment to develop and implement standards for AI system evaluation and testing.\n\nAdditionally, the summit also agreed on the following key points:\n\n* Each country will invest in public sector capabilities to test and evaluate AI systems.\n* Advanced AI systems developers will take responsibility for ensuring the safety of their systems.\n* A global network of research institutions, governments, and industries will be established to share knowledge and best practices on AI safety.\n\nIt is worth noting that these points were agreed upon by 28 countries participating in the summit.', "I don't know.", "I don't know what the MMLU benchmark is or how it evaluates AI models like Tongyi Qianwen 2.0 for generative AI. The provided context only mentions Alibaba Cloud's launch of Tongyi Qianwen 2.0 and its plans to open-source a model with 720 billion parameters, but does not mention the MMLU benchmark.", "I don't know how the MMLU benchmark assesses AI models like Tongyi Qianwen 2.0. The provided context only mentions Alibaba Cloud's launch of Tongyi Qianwen 2.0 and its plans to open-source it, but does not mention the MMLU benchmark or its assessment method.", "I don't know how changes in AI evaluation criteria would specifically affect Large Language Models (LLMs). The provided context discusses the framework for evaluating Artificial General Intelligence (AGI) and its implications, but it doesn't directly address the impact of changes in AI evaluation criteria on LLMs. If you have more specific information or context about the changes in AI evaluation criteria, I might be able to provide a more informed answer.", 'The provided context mentions several factors related to responsible AI deployment and use, but it does not explicitly mention "ethical AI innovation" as a specific topic.\n\nHowever, some relevant points can be inferred:\n\n1. The document emphasizes the importance of creating a strategy for responsible AI deployment and use in the healthcare sector.\n2. It highlights the need to promote trustworthy AI development and use, with a focus on protecting consumers and workers.\n3. The Executive Order on the Safe, Secure, and Trustworthy Development and Use of Artificial Intelligence (E.O. 14110) mentions principles for ensuring responsible AI innovation.\n\nWhile these points touch on aspects of responsible AI development, they do not explicitly address all factors that might be relevant to "ethical AI innovation." Therefore, I don\'t know the answer to this question based on the provided context.']

```

---

<small>

* before

* `print(answer)`

<small> <small> <small>

```python

    ['According to the retrieved context, TechRepublic has highlighted that Samsung has developed a generative AI called "Samsung Gauss" which is expected to compete with Meta\'s Llama2 and Google\'s Assistant on recent smartphones.', 'According to the retrieved context, the dates and location for CES 2024 are:\n\n* Dates: 2024.1.9~12\n* Location: 미국 (United States), 라스베가스 (Las Vegas)', 'The key aspects of AI global cooperation highlighted in the G7\'s approach to ethical considerations and regulatory frameworks for advanced AI systems include:\n\n1. International Code of Conduct for Advanced AI Systems: The G7 has agreed on an international code of conduct for advanced AI systems, which includes provisions for identifying and mitigating risks, ensuring transparency and accountability, and promoting responsible AI development and use.\n2. Voluntary Adoption: The G7 is encouraging voluntary adoption of the international code of conduct by companies developing advanced AI systems.\n3. Risk Evaluation and Mitigation: The G7 is requiring a risk evaluation and mitigation process for advanced AI systems, including the identification of potential risks and the implementation of measures to mitigate them.\n4. Transparency and Accountability: The G7 is promoting transparency and accountability in AI development and use, including the disclosure of information about AI systems and their limitations.\n5. Cooperation with International Organizations: The G7 is encouraging cooperation with international organizations, such as the United Nations, to promote global coordination on AI issues.\n6. Development of a Global Framework: The G7 is working towards developing a global framework for regulating advanced AI systems, which would provide a common set of standards and guidelines for countries to follow.\n7. Collaboration with Other Countries: The G7 is engaging in collaboration with other countries, including through the "Hyorishima AI Process", to promote global cooperation on AI issues.\n\nThese aspects highlight the G7\'s commitment to promoting responsible AI development and use, while also recognizing the need for international cooperation and coordination to address the challenges posed by advanced AI systems.', 'I don\'t know the specific answer to your question based on the provided context. However, I can suggest that you may want to look into the documents related to "보건복지 부문" (Health and Welfare Department) or "혁신과 경쟁 촉진" (Innovation and Competition Promotion) sections in the PDF file, as they seem to be related to AI development and deployment strategies.', "The main takeaways from the AI Safety Summit on global AI testing include:\n\n1. The importance of cooperation among nations, international organizations, companies, civil society, and academia in ensuring AI safety.\n2. The need for transparency and accountability in AI development, particularly from advanced AI systems developers.\n3. The agreement to develop and implement standardized evaluation methods and test tools for AI systems.\n4. The commitment to enhance public awareness and understanding of AI risks and benefits through education and outreach initiatives.\n5. The establishment of a global framework for testing and evaluating AI systems, with the involvement of governments, research institutions, and industry partners.\n\nAdditionally, it was announced that the UK would lead the development of a new AI safety test plan, which would be conducted by the UK's AI Safety Research Institute, in collaboration with other countries.", "I don't know. The provided context does not mention any link between data provenance, model efficiency, and NLP accuracy.", "I don't know what the MMLU benchmark is or how it evaluates AI models like Tongyi Qianwen 2.0 for generative AI. The provided context only mentions Alibaba Cloud's launch of Tongyi Qianwen 2.0 and its plans to open-source a model with 720 billion parameters, but does not mention the MMLU benchmark.", "I don't know. The provided context doesn't mention the MMLU benchmark or how it assesses AI models like Tongyi Qianwen 2.0. It only discusses the classification framework for AGI proposed by Google DeepMind and its relation to AI development and use.", "I don't know how changes in AI evaluation criteria would specifically affect Large Language Models (LLMs). The provided context discusses the framework for evaluating Artificial General Intelligence (AGI) and its implications, but it does not directly address the impact of changes in AI evaluation criteria on LLMs.", 'According to the provided context, the following factors are mentioned as being important for ethical AI innovation:\n\n1. 책임 있는 AI 배포와 사용을 위한 전략 (strategies for responsible AI deployment and use)\n2. 소비자 보호와 근로자 지원 (protection of consumers and workers)\n3. 맞춤형 개인교습 등 학교 내 AI 교육 도구 관련 자원 개발 (development of tailored education resources for schools)\n4. AI로 인한 근로자 피해를 완화하고 이점을 극대화하는 원칙 (principles to mitigate worker harm and maximize benefits from AI)\n5. 혁신과 경쟁 촉진 (innovation and competition promotion)\n\nThese factors are mentioned in the context of Executive Order 14110, which aims to promote safe, secure, and trustworthy development and use of artificial intelligence.']

```

---

* **`LLM`이 생성한 답변 = `answer` 컬럼에 저장하기**

In [ ]:
# 'answer' 컬럼 덮어쓰기 또는 추가

if "answer" in test_dataset.column_names:
    test_dataset = test_dataset.remove_columns(["answer"]).add_column("answer", answer)

else:
    test_dataset = test_dataset.add_column("answer", answer)

In [ ]:
print(test_dataset)

<small>

* **`print(test_dataset)`**

```python

        Dataset({
            features: ['question', 'contexts', 'ground_truth', 'evolution_type', 'metadata', 'episode_done', 'answer'],
            num_rows: 10
        })

```

In [ ]:
print(type(test_dataset))                       # <class 'datasets.arrow_dataset.Dataset'>

---

#### **2) `답변 평가`**

##### **`➀ Context Recall`**

* **`검색된 context`가 `LLM`이 생성한 `답변`과 `얼마나 일치`하는지를 측정**

  * `question`, `ground truth` 및 `검색된 context` → 계산

  * 값 = `0 ~ 1` 사이 → **`높을수록 더 나은 성능`**

* ![context_recall](../15_Evaluations/assets/02_context_recall.png)

  * `Ground truth` 답변에서 `context recall`을 추정하기 위해, `ground truth` 답변의 `각 주장이 검색된 context`에 `귀속`될 수 있는지 분석

  * 이상적인 시나리오: `ground truth 답변`의 `모든 주장`이 `검색된 context`에 `귀속`될 수 있어야 함

##### **`➁ Context Precision`**

* **`contexts` 내의 `ground-truth` `관련 항목`들이 `상위 순위`에 있는지를 평가하는 지표**

  * 이상적: `모든 관련 chunks` = `상위 순위`에 나타나야 함 

  * `question`, `ground_truth`, `contexts` → 계산

  * 값 = **`0 ~ 1` 사이** → **`높은 점수일수록 더 나은 정밀도`**

* 게산식

  * `Context Precision@K`의 계산식

  * ![Context Precision@K](../15_Evaluations/assets/02_context_precision.png)

<br>

* 
  * `Precision@k`의 계산식

  * ![Precision@k](../15_Evaluations/assets/02_precision.png)

    * `k` = **`contexta`의 총 `chunk` 수**

    * `vₖ ∈ {0, 1}` = 순위 k에서의 관련성 지표

<br>

* 정보 검색 시스템에서 검색된 컨텍스트의 품질을 평가하는 데 사용

  * 관련 정보가 얼마나 `정확하게 상위 순위`에 `배치`되었는지를 측정 → 시스템의 성능 판단

##### **`➂ Answer Relevancy`**

* **`생성된 답변`이 주어진 `prompt`에 `얼마나 적절한지`를 `평가`하는 지표** 

* 주요 특징

  * 목적: 생성된 `답변`의 `관련성 평가`

  * 점수 해석: 
    * 낮은 점수 = `불완전`하거나 `중복 정보`를 `포함`한 답변
    * 높은 점수 = `더 나은 관련성`을 나타냄

  * 계산에 사용되는 요소: `question`, `context`, `answer`

* 게산 방법

  * 원래 `question`과 `answer`를 기반으로 생성된 `인공적인 질문들 간`의 `평균 코사인 유사도`로 정의

  * 수식 ➀

    * ![answer_relevancy_1](../15_Evaluations/assets/02_answer_relevancy_1.png)

  * 수식 ➁

    * ![answer_relevancy_2](../15_Evaluations/assets/02_answer_relevancy_2.png)

  * 참고

    * **`E₀`** = 생성된 질문 `i`의 임베딩

    * **`Eᵍᵢ`** ($E_{g_i}$) = 원래 질문의 임베딩

    * ***`N`*** = 원래 질문의 임베딩 (*기본값 = 3*)

    * 주의사항: 실제로는 점수가 대부분 0과 1 사이 → 코사인 유사도의 특성상 수학적으로 -1에서 1 사이의 값을 가질 수도 있음

<br>

* 질문-답변 시스템의 성능을 평가하는 데 유용

  * `생성된 답변`이 `원래 질문의 의도`를 얼마나 잘 `반영`하는지를 측정

##### **`➃ Faithfulness`**

* **`생성된 답변의 사실적 일관성을 주어진 컨텍스트와 비교하여 측정하는 지표`**

* 주요 특징

  * 목적: `답변`의 `사실적 일관성`을 `컨텍스트`와 `비교` → 평가

  * 계산 요소: `답변`과 `검색된 컨텍스트` 사용

  * 점수 범위: `0 ~ 1 사이`로 조정 → **`높을수록 더 좋음`**

* 계산 방법

  * 수식

  * ![faithfulness](../15_Evaluations/assets/02_faithfulness.png)

<br>

* 
  * 계산 과정 ➀: `생성된 답변`에서 `주장`(claims)들을 `식별`

  * 계산 과정 ➁: `각 주장`을 주어진 `컨텍스트`와 `대조` `검증` → `컨텍스트`에서 `추론 가능`한지 `확인`

  * 계산 과정 ➂: 위 `수식`을 사용하여 `점수 계산`

<br>

* 
  * 예시

    * 질문: **`"아인슈타인은 어디서, 언제 태어났나요?"`**

    * 컨텍스트: **`"알버트 아인슈타인(1879년 3월 14일 출생)은 독일 출신의 이론 물리학자로, 역사상 가장 위대하고 영향력 있는 과학자 중 한 명으로 여겨집니다."`**

    * **`높은`** 충실도 답변: **`"아인슈타인은 1879년 3월 14일 독일에서 태어났습니다."`**

    * *`낮은`* 충실도 답변: *`"아인슈타인은 1879년 3월 20일 독일에서 태어났습니다."`*

<br>

* `생성된 답변`이 주어진 `컨텍스트`에 얼마나 `충실`한지를 `평가`하는 데 유용
  * 특히 `질문-답변` 시스템의 `정확성`과 `신뢰성`을 측정하는 데 중요

In [ ]:
from ragas import evaluate

from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)

print("✅ 모험 평가 함수 생성 완료")


# -----------------------------------------------------------
# 2. 모든 Ragas Metrics에 LangChain LLM 객체 'llm' 직접 주입
# -----------------------------------------------------------

from langchain_google_genai import ChatGoogleGenerativeAI

from dotenv import load_dotenv
import os 

# API 키 확인
if not os.getenv("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = input("Enter your Google API key: ")    

# gemini LLM 정의
llm2 = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    temperature=0,
    google_api_key=os.getenv("GOOGLE_API_KEY")
    )

print("✅ gemini Critic LLM 정의 완료. (모델: gemini-2.5-flash-lite) ")

result = evaluate(
    dataset=test_dataset,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
    llm=llm2,
)

print()
print("="*60)
print("✅ 평가 완료!")
print("="*60)
print()

<small>

* ✅ 모험 평가 함수 생성 완료 - (`0.2s`)

* ✅ Ollama Critic LLM 정의 완료. (모델: llama3.2:3b, JSON 출력 강제)

* ✅ Ragas Metrics에 LLM 주입 완료. 이제 평가를 시작할 준비가 되었습니다.

<small>

* ✅ 모험 평가 함수 생성 완료 - (`0.5s`)

* ✅ gemini Critic LLM 정의 완료. (모델: gemini-2.5-flash-lite) 

* ✅ Ragas Metrics에 LLM 주입 완료. 이제 평가를 시작할 준비가 되었습니다.

In [ ]:
# -----------------------------------------------------------
# 데이터셋 첫번째 로우 확인하기
# -----------------------------------------------------------

try:
    print("\n--- 1. 데이터셋 전체 컬럼 이름 확인 ---")
    
    # 데이터셋의 실제 컬럼 이름 출력
    actual_columns = test_dataset.column_names
    print(f"실제 컬럼 목록: {actual_columns}")

    # Ragas 필수 컬럼의 포함 여부 확인
    required_cols = ['question', 'answer', 'contexts', 'ground_truth']
    missing_cols = [col for col in required_cols if col not in actual_columns]
    
    if missing_cols:
        print(f"\n❌ [치명적 오류]: Ragas 필수 컬럼 중 다음이 누락되었습니다: {missing_cols}")
    else:
        print("\n✅ [컬럼 이름 확인]: Ragas 필수 컬럼이 모두 존재합니다.")

    print("\n--- 2. 첫 번째 로우의 값 및 타입 확인 (가장 중요) ---")
    first_row = test_dataset[0]

    for col in required_cols:
        value = first_row.get(col)
        value_type = type(value)

        # contexts 컬럼에 대한 특별 검사
        if col == 'contexts':
            # contexts가 리스트인지 확인
            if value_type is list:
                # 리스트의 첫 번째 요소가 문자열인지 확인
                inner_type = type(value[0]) if value and len(value) > 0 else 'Empty List'
                print(f"  - {col} (타입: {value_type}, 내부 타입: {inner_type})")
                if inner_type != str:
                    print("    🚨 경고: contexts 내부 요소는 반드시 문자열(str)이어야 합니다.")
            else:
                print(f"  - {col} (타입: {value_type})")
                print("    ❌ 치명적 오류: contexts 컬럼은 반드시 리스트(list)여야 합니다.")
        else:
            # question, answer, ground_truth는 문자열인지 확인
            print(f"  - {col} (타입: {value_type})")
            if value_type is not str:
                print(f"    🚨 경고: {col} 컬럼은 반드시 문자열(str)이어야 합니다.")

except NameError:
    print("\n❌ 오류: 'test_dataset'이 정의되지 않았습니다. 데이터셋 로드를 먼저 실행해 주세요.")
except Exception as e:
    print(f"\n❌ 진단 중 예상치 못한 오류 발생:")
    print(e)

In [ ]:
# -----------------------------------------------------------
# 3. 평가 실행
# -----------------------------------------------------------

print("평가 점수:")
for metric, score in result.items():
    print(f"  {metric}: {score:.4f}")

# DataFrame 변환
result_df = result.to_pandas()
print()
print("결과 DataFrame:")
print(result_df.head())

# 저장
result_df.to_csv("../15_Evaluations/data/ragas_evaluation_result_2.csv", index=False)
print()
print("✅ 저장 완료: data/ragas_evaluation_result.csv")

<small>

* **`try_1` → ❌** - (`9m 1.7s`)

```bash

        Evaluating:   100%|████████████████████████████████████████| 40/40 [09:00<00:00, 17.15s/it]

```

```python

        {'context_precision': 1.0000, 'faithfulness': nan, 'answer_relevancy': nan, 'context_recall': 1.0000}

```

---

In [ ]:
# 데이터프레임 형태로 확인하여 nan 발생 여부 확인
result_df = result.to_pandas()
result_df.head()

In [ ]:
result_df.loc[:, "context_precision":"context_recall"]

---

* next: ***`03. 생성한 평가용 데이터셋 업로드 (HuggingFace Dataset)`***

---